In [1]:
head = 'l30_r15'
directory = '../data/'
model_fname ='../models/Logistic + L2 (C=1.00) norm l30_r15.pkl'


import pickle    
from py.utils import load_data

x, y, x_words, vocabs = load_data(head, directory)
with open(model_fname, 'rb') as f:
    classifier = pickle.load(f)

x shape = (15715, 4551)
y shape = (15715,)
# features = 4551
# L words = 15715


In [2]:
coefficient = {vocabs[j]:coef for j, coef in enumerate(classifier.coef_[0])}

In [8]:
list(coefficient.items())[:5]

[('어오고', -0.0035026965225787533),
 ('어놓는', -0.0017447214860182797),
 ('느냐고', -0.027019952035748179),
 ('라느니', 0.0013030948275454116),
 ('으셔서', -0.3228328145095205)]

In [5]:
import sys
from soynlp.utils import get_process_memory
import numpy as np

In [186]:
class TrainedNounExtractor:
    def __init__(self, coefficient, max_length=8):
        self._coef = coefficient
        self.lmax = max_length
        
    def extract(self, sents, min_count=10, min_noun_score=0.1):
        self.lrgraph, self.lset, self.rset = self._build_lrgraph(sents, min_count)
        self.lentropy, self.rentropy = self._branching_entropy(lrgraph)
        scores = self._compute_noun_score(self.lrgraph)
        return scores, self.lrgraph
        scores = self._postprocessing(lrgraph, scores)

    def _build_lrgraph(self, sents, min_count, pruning_min_count=2):
        lset = {}
        rset = {}
        for n_sent, sent in enumerate(sents):
            for eojeol in sent.split():
                for e in range(1, min(len(eojeol), self.lmax)+1):
                    l = eojeol[:e]
                    r = eojeol[e:]
                    lset[l] = lset.get(l, 0) + 1
                    rset[r] = rset.get(r, 0) + 1
            if n_sent % 1000 == 999:
                args = (n_sent+1, len(lset), len(rset), get_process_memory())
                sys.stdout.write('\rscaning vocabulary ... %d sents #(l= %d, r= %d), mem= %.3f Gb' % args)
            if n_sent % 500000 == 499999:
                lset = {l:f for l,f in lset.items() if f >= pruning_min_count}
                rset = {l:f for l,f in rset.items() if f >= pruning_min_count}
        lset = {l:f for l,f in lset.items() if f >= min_count}
        rset = {l:f for l,f in rset.items() if f >= min_count}
        
        n_sents = n_sent
        
        lrgraph = {}
        for n_sent, sent in enumerate(sents):
            for eojeol in sent.split():
                for e in range(1, min(len(eojeol), self.lmax)+1):
                    l = eojeol[:e]
                    r = eojeol[e:]
                    if not (l in lset) or not (r in rset):
                        continue
                    rdict = lrgraph.get(l, {})
                    rdict[r] = rdict.get(r, 0) + 1
                    lrgraph[l] = rdict            
            if n_sent % 1000 == 999:
                args = (100*(n_sent+1)/n_sents, '%', n_sent+1, n_sents, get_process_memory())
                sys.stdout.write('\rbuilding lrgraph ... (%.3f %s, %d in %d), mem= %.3f Gb' % args)
        args = (len(lset), len(rset), sum((len(rdict) for rdict in lrgraph.values())), get_process_memory())
        print('\rlrgraph has been built. (#L= %d, #R= %d, #E=%d), mem= %.3f Gb' % args)
        return lrgraph, lset, rset
    
    def _branching_entropy(self, lrgraph):
        from collections import defaultdict
        def entropy(d):
            sum_ = sum(d.values())
            if sum_ == 0: return 0
            return -1 * sum((v/sum_) * np.log(v/sum_) for v in d.values())
        def branch_map(d, get_branch):
            b = defaultdict(lambda: 0)
            for ext,f in d.items():
                if ext != '':
                    b[get_branch(ext)] += f
            return b
        def all_entropy(graph, get_branch=lambda x:x[0]):
            return {w:entropy(branch_map(d, get_branch)) for w, d in graph.items()}
        def to_rlgraph(lrgraph):
            rlgraph = defaultdict(lambda: defaultdict(lambda: 0))
            for l, rdict in lrgraph.items():
                for r, f in rdict.items():
                    rlgraph[r][l] += f
            return {r:dict(ldict) for r, ldict in rlgraph.items()}    
        print('compute branching entropy ...', end='')
        lentropy = all_entropy(lrgraph)
        rentropy = all_entropy(to_rlgraph(lrgraph), get_branch=lambda x:x[-1])
        print(' done')
        return lentropy, rentropy
        
    def _compute_noun_score(self, lrgraph):
        from collections import namedtuple
        Score = namedtuple('Score', 'score frequency branching_entropy feature_fraction eojeol_fraction')
        scores = {}
        n = len(lrgraph)
        for i, (l, rdict) in enumerate(lrgraph.items()):
            rdict_ = {r:f for r,f in rdict.items() if r in self._coef}
            rsum = sum((f for r,f in rdict.items() if r != ''))
            frequency = rsum + rdict.get('', 0)
            feature_fraction = sum(rdict_.values()) / rsum if rsum > 0 else 0
            eojeol_fraction = 1 - rsum / frequency
            if not rdict_:
                score = 0
            else:
                score = sum(f*self._coef[r] for r, f in rdict_.items()) / sum(rdict_.values())
            scores[l] = Score(score, frequency, self.lentropy.get(l, 0), feature_fraction, eojeol_fraction)
            if (i+1) % 1000 == 0:
                args = (100*(i+1)/n, '%', i+1, n)
                sys.stdout.write('\rcompute noun score ... (%.3f %s, %d in %d)' % args)
        print('\rcomputing noun score has been done.')
        return sorted(scores.items(), key=lambda x:x[1].score, reverse=True)
        
    def _postprocessing(self, lrgraph, scores):
        print('hi')

In [187]:
from config import sentence_fname
class Sentences:
    def __init__(self, fname):
        self.fname = fname
    def __iter__(self):
        with open(self.fname, encoding='utf-8') as f:
            for doc in f:
                yield doc.strip()

sentences = Sentences(sentence_fname)
noun_extractor = TrainedNounExtractor(coefficient)
scores, lrgraph = noun_extractor.extract(sentences)

lset, rset = noun_extractor.lset, noun_extractor.rset

lrgraph has been built. (#L= 201559, #R= 86324, #E=2487566), mem= 2.063 Gb
compute branching entropy ... done
computing noun score has been done.


In [219]:
def to_rlgraph(lrgraph):
    from collections import defaultdict
    rlgraph = defaultdict(lambda: defaultdict(lambda: 0))
    for l, rdict in lrgraph.items():
        for r, f in rdict.items():
            rlgraph[r][l] += f
    return {r:dict(ldict) for r, ldict in rlgraph.items()}

rlgraph = to_rlgraph(lrgraph)

## Preprocessor, Postprocessing의 필요성

- lrgraph를 만들 때 한글 외의 기호를 삭제한 어절로 이뤄져야 함 
- N = Nsub + J problem
    - 중세보편주 + {의, 의의}
- [N+Jsub] + J problem
    - 대학생과 + {'', 의}
    - {의, 과의} 모두 조사


In [204]:
# useful function
def pretty(namedtuple_instance, end='\n'):
    print('%s(%s)' % (namedtuple_instance.__class__.__name__, ', '.join(['%s=%d' % (field, value) if (type(value) == int or type(value) == np.int) else '%s=%.3f' % (field, value)  for field, value in namedtuple_instance._asdict().items()])),end=end )

def pretty_list(l):
    print('[', end='')
    for w, nt in l[:10]:
        print('(%s,'%w, end=' ')
        pretty(nt, end='),\n')
    print(']' if len(l) <= 10 else '... ]')

In [188]:
lrgraph['중세보편주']

{'의': 6, '의를': 7, '의의': 2}

In [189]:
lrgraph['대학생과']

{'': 24, ',': 1, '의': 1}

In [190]:
sorted(lrgraph['대학생'].items(), key=lambda x:x[1], reverse=True)[:10]

[('', 165),
 ('이', 74),
 ('들이', 73),
 ('들의', 67),
 ('들은', 36),
 ('의', 26),
 ('을', 26),
 ('과', 24),
 ('들을', 23),
 ('은', 21)]

In [202]:
pretty_list(list(filter(lambda x:x[1].frequency > 100 and len(x[0]) > 1, scores))[:10])

[(개인과, Score(score=6.211, frequency=124, branching_entropy=0.562, feature_fraction=0.750, eojeol_fraction=0.968)),
(로버트, Score(score=6.211, frequency=175, branching_entropy=0.562, feature_fraction=0.250, eojeol_fraction=0.977)),
(자유주, Score(score=6.211, frequency=182, branching_entropy=-0.000, feature_fraction=0.148, eojeol_fraction=0.000)),
(한낱, Score(score=6.211, frequency=127, branching_entropy=-0.000, feature_fraction=1.000, eojeol_fraction=0.984)),
(우르, Score(score=6.211, frequency=158, branching_entropy=0.651, feature_fraction=0.006, eojeol_fraction=0.006)),
(그런대로, Score(score=6.211, frequency=188, branching_entropy=0.693, feature_fraction=0.500, eojeol_fraction=0.989)),
(당신과, Score(score=6.211, frequency=128, branching_entropy=0.349, feature_fraction=0.889, eojeol_fraction=0.930)),
(낭만주, Score(score=6.211, frequency=179, branching_entropy=-0.000, feature_fraction=0.223, eojeol_fraction=0.000)),
(여지껏, Score(score=6.211, frequency=106, branching_entropy=0.637, feature_fraction=0.3

In [203]:
pretty_list(list(scores)[:10])

[(좌익과, Score(score=6.211, frequency=17, branching_entropy=-0.000, feature_fraction=1.000, eojeol_fraction=0.824)),
(과거부터, Score(score=6.211, frequency=22, branching_entropy=-0.000, feature_fraction=1.000, eojeol_fraction=0.955)),
(<시인, Score(score=6.211, frequency=17, branching_entropy=0.693, feature_fraction=0.500, eojeol_fraction=0.882)),
(負, Score(score=6.211, frequency=3, branching_entropy=-0.000, feature_fraction=1.000, eojeol_fraction=0.000)),
(<밤, Score(score=6.211, frequency=4, branching_entropy=0.562, feature_fraction=0.250, eojeol_fraction=0.000)),
(각권, Score(score=6.211, frequency=17, branching_entropy=-0.000, feature_fraction=1.000, eojeol_fraction=0.941)),
(개인과, Score(score=6.211, frequency=124, branching_entropy=0.562, feature_fraction=0.750, eojeol_fraction=0.968)),
(자와, Score(score=6.211, frequency=82, branching_entropy=0.796, feature_fraction=0.714, eojeol_fraction=0.915)),
(진료와, Score(score=6.211, frequency=11, branching_entropy=-0.000, feature_fraction=1.000, eojeol_

## Dev: post-processing

### branching entropy는 좀 구해볼까? 
    
    L, right-side 
    R, left-side
    
    ## Complete
    
    >>> for w in ['대학', '대학생', '대학생과']:
    >>>     print(w, '%.3f' % lentropy.get(w, 0))

    대학 2.832
    대학생 1.721
    대학생과 0.000

In [214]:
min_noun_score = 0.5

In [205]:
score_dict = dict(scores)
    
pretty(score_dict['떡볶'])
pretty(score_dict['떡볶이'])

Score(score=1.104, frequency=67, branching_entropy=0.366, feature_fraction=0.761, eojeol_fraction=0.000)
Score(score=2.978, frequency=64, branching_entropy=2.335, feature_fraction=0.739, eojeol_fraction=0.281)


### Nsub + J: 떡볶 + 이

    f(떡볶) ~= f(떡볶이): drop-rate, branching entropy로 하자
    (떡볶이 in Noun) and (이 in Josa)
    주로 한글자 짜리 조사가 문제 

In [217]:
josa = sorted(filter(lambda x:x[1] > 0, coefficient.items()), key=lambda x:x[1], reverse=True)
len(josa)

2375

### L2 는 정말 좋은 Regularization 인가? 

L2는 coefficients의 제곱의 합을 최소화 하려고 노력하기 때문에 자주 이용되는 features에 대한 coefficient의 절대값을 키우려는 경향이 있다. 

"- 해줌으로써"는 명사를 구분할 수 있는 좋은 조사이지만, 데이터에 등장한 횟수는 별로 많지 않다. 그렇기 때문에 coefficient가 매우 작게 나오게 된다. (coef = 0.000196)

반대로 "- 하는"의 경우, [좋아 - 하는, 못 - 하는, ...]등의 오류가 있음에도 압도적인 명사들 때문에 높은 coefficient (2.033)을 가지게 된다. 

이런 입장에서 오히려 L2 regularization Logistic Regression 보다 Naive Bayes나 proportion 기반 방법이 더 좋은 것이 아닌가? 

L2 regularization은 학습데이터에서 등장한 R set의 frequency에 scale의 영향을 받는다. 본질적인 이유는 L-R matrix가 sparse matrix여서 L을 설명하는 R set features가 언제나 이용되는 것이 아니며, R set frequency 역시 Zipw's law를 따라 skewed 되어있기 때문이다. 그리고 데이터셋이 바뀜에 따라 R set frequency distribution이 달라지면 coefficient scale이 문제가 될 수 있다. 세종말뭉치에 이용되었던 문서들과 현제의 온라인 문서에서의 R set distribution은 분명 다를 수 밖에 없고, 지금 질 좋은 features가 될 수 있는 R set 들의 coefficients의 절대값이 작다면 분석이 잘 되지 않을 수도 있다. 

In [263]:
w = '해줌으로써'
print(str(sorted(rlgraph.get(w, {}).items(), key=lambda x:x[1], reverse=True)[:5]) + ' - %s' % w)
str(sorted(rlgraph.get(w, {}).items(), key=lambda x:x[1], reverse=True)[:20])[:-1] + (']' if len(rlgraph.get(w, {})) <= 20 else ', ...]')

[('보장', 2), ('마련', 2), ('보상', 2), ('명시', 1), ('보전', 1)] - 해줌으로써


"[('보장', 2), ('마련', 2), ('보상', 2), ('명시', 1), ('보전', 1), ('지원', 1), ('얘기', 1), ('지지', 1), ('제공', 1), ('대출', 1), ('인정', 1), ('대여', 1), ('확인', 1)]"

In [268]:
w = '하는'
print(str(sorted(rlgraph.get(w, {}).items(), key=lambda x:x[1], reverse=True)[:5]) + ' - %s' % w)
str(sorted(rlgraph.get(w, {}).items(), key=lambda x:x[1], reverse=True)[:20])[:-1] + (']' if len(rlgraph.get(w, {})) <= 20 else ', ...]')

[('못', 2114), ('말', 1953), ('생각', 1356), ('좋아', 1002), ('원', 847)] - 하는


"[('못', 2114), ('말', 1953), ('생각', 1356), ('좋아', 1002), ('원', 847), ('사용', 834), ('사랑', 673), ('일', 647), ('요구', 598), ('존재', 551), ('추구', 528), ('주장', 502), ('해당', 501), ('발생', 487), ('차지', 484), ('제공', 481), ('등장', 393), ('이해', 383), ('의미', 376), ('시작', 374), ...]"

In [269]:
coefficient['하는'], coefficient['해줌으로써'], 

(2.0333080567632096, 0.00019593425186685041)

In [270]:
josa[:20]

[('의', 6.2108250932929217),
 ('를', 5.2275917393592097),
 ('가', 4.3986751160523632),
 ('한', 3.7657194150629776),
 ('에', 3.5512693551084609),
 ('로', 3.5400023136261112),
 ('이', 3.2809566440846751),
 ('을', 2.8668114740415227),
 ('으로', 2.6465314028770761),
 ('적인', 2.4873089663273582),
 ('하고', 2.3397713254276393),
 ('된', 2.1779089976672568),
 ('에서', 2.1476524916015003),
 ('적', 2.1083448239349738),
 ('하는', 2.0333080567632096),
 ('할', 2.0317829273285861),
 ('적으로', 1.6583744760803698),
 ('이다', 1.5285098649962987),
 ('들은', 1.5025060153070737),
 ('에게', 1.4847704268894344)]

In [257]:
josa[-200:-180]

[('됐음을', 0.00020547991455129269),
 ('끓는', 0.00020398189911457259),
 ('냐구요', 0.00020381950100549124),
 ('이었구나', 0.00020337780768687254),
 ('인가라고', 0.00020283424143817397),
 ('했느냐는', 0.00020240669293992069),
 ('에서두', 0.00020064144022782202),
 ('했단다', 0.00020015317375592496),
 ('적자금을', 0.00019608633897839539),
 ('해줌으로써', 0.00019593425186685041),
 ('에서만은', 0.00019590588147109036),
 ('달러', 0.00019551343979462479),
 ('해줘', 0.00019425354394409135),
 ('하셨고', 0.00019384787685696134),
 ('싶은', 0.00018758541793060106),
 ('했었지', 0.00018672308447793918),
 ('이었음은', 0.00018470679938860485),
 ('하셨는데', 0.00018294591009844179),
 ('하니까요', 0.00018007395979813116),
 ('이서만', 0.00017946260198717647)]

### [N + Jsub] + J: 대학생과 + 의

    f(대학생) >> f(대학생과)
    Right side branching entropy(대학생) ~= high
    (과의 in Josa) and (대학생 in Noun)

### Compound: 소수 + [집단 + 의]

    소수 + 집단의 (집단의 = 집단 + 의 인지 확인)
    Noun score 대체
